In [1]:
!pip install --quiet -U sentence-transformers

In [2]:
# Dowloading split dataset
! wget https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/quora-IR-dataset.zip
! unzip ./quora-IR-dataset.zip

--2022-04-21 20:45:53--  https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/quora-IR-dataset.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93570055 (89M) [application/zip]
Saving to: ‘quora-IR-dataset.zip’

quora-IR-dataset.zi 100%[===================>]  89.24M  20.9MB/s    in 8.3s    

2022-04-21 20:46:02 (10.8 MB/s) - ‘quora-IR-dataset.zip’ saved [93570055/93570055]

Archive:  ./quora-IR-dataset.zip
   creating: information-retrieval/
  inflating: information-retrieval/corpus.tsv  
  inflating: information-retrieval/dev-queries.tsv  
  inflating: information-retrieval/test-queries.tsv  
  inflating: quora_duplicate_questions.tsv  
   creating: classification/
  inflating: classification/dev_pairs.tsv  
  infla

In [3]:
import os
from datetime import datetime
import csv
import random
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, evaluation
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder import CrossEncoder

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


# setting seed
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

## Evaluating SBERT models

In [4]:
# test_sentences1 = []
# test_sentences2 = []
# test_labels = []
# with open("classification/test_pairs.tsv", encoding='utf8') as fIn:
#     reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
#     for row in reader:
#         test_sentences1.append(row['question1'])
#         test_sentences2.append(row['question2'])
#         test_labels.append(int(row['is_duplicate']))
        
# evaluator = evaluation.BinaryClassificationEvaluator(test_sentences1, test_sentences2, test_labels, 
#                                                      batch_size=32, 
#                                                      write_csv=True, 
#                                                      show_progress_bar=True)

In [5]:
# # model_name = "../input/qqp-sbert-base/output/sbert_base_mnr_16" # base SBERT model
# # model_name = "../input/aug-sbert-stsb/output/sbert_stsb_mnr_16" # stsb cross-encoder + SBERT model
# model_name = "../input/qqp-sbert-custom/output/sbert_custom_mnr_16" # quora cross-encoder + SBERT model

# model = SentenceTransformer(model_name)

# model.evaluate(evaluator, output_path='sbert_quora_eval')

In [6]:
# eval_df = pd.read_csv('base_sbert_eval/binary_classification_evaluation_results.csv')
# eval_df

## Evaluating cross-encoder model

In [7]:
test_samples = []
with open('classification/test_pairs.tsv', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        sample = InputExample(texts=[row['question1'], row['question2']], label=int(row['is_duplicate']))
        test_samples.append(sample)
        
test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_samples)

In [8]:
model = CrossEncoder('../input/qqp-cross-base/output/cross_base_mnr_16')
test_evaluator(model, output_path='')

0.8788630974325571

In [9]:
eval_df = pd.read_csv('CEBinaryClassificationEvaluator_results.csv')
eval_df

,epoch,steps,Accuracy,Accuracy_Threshold,F1,F1_Threshold,Precision,Recall,Average_Precision
0,-1,-1,0.849923,0.208017,0.822477,0.114806,0.770956,0.881376,0.878863
